
# 02 — EDA & Cleaning (Base)

**Tujuan notebook ini**  
- Memuat *sample* hasil langkah 01.  
- Melakukan *scan* cepat tipe & missing.  
- Menangani nilai sentinel umum (mis. `DAYS_EMPLOYED == 365243`).  
- Menambah *feature* rasio inti (kemampuan bayar).  
- Menyimpan dataset **interim** yang sudah dibersihkan dasar (`data/interim/...`) untuk dipakai modeling.
  
> Catatan: **Imputasi & encoding final** akan dilakukan di **notebook modeling (03)** via `sklearn` pipeline (agar tidak *data leakage*).


In [2]:

from pathlib import Path
import os


MY_DATA_DIR = Path("C:/Users/felix/hci_final_project/data")


DATA_DIR = MY_DATA_DIR if MY_DATA_DIR.exists() else Path("data")

SAMPLE_PATH = DATA_DIR / "sample" / "application_train_sample.csv"
RAW_PATH    = DATA_DIR / "raw" / "application_train.csv"   # opsional (untuk full run nanti)

TARGET = "TARGET"
ID_COL = "SK_ID_CURR"

print("DATA_DIR :", DATA_DIR.resolve())
print("SAMPLE   :", SAMPLE_PATH)
print("Exists?  :", SAMPLE_PATH.exists())


DATA_DIR : C:\Users\felix\hci_final_project\data
SAMPLE   : C:\Users\felix\hci_final_project\data\sample\application_train_sample.csv
Exists?  : True


In [3]:

import pandas as pd


use_path = SAMPLE_PATH if SAMPLE_PATH.exists() else RAW_PATH
df = pd.read_csv(use_path, low_memory=False)
print(df.shape)
df.head(3)


(12387, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,333721,0,Cash loans,M,Y,Y,1,292500.0,1102500.0,32364.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,110157,0,Revolving loans,F,N,Y,0,270000.0,720000.0,36000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0
2,276815,0,Revolving loans,F,Y,Y,1,135000.0,360000.0,18000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:


meta = []
for c in df.columns:
    s = df[c]
    meta.append({
        "column": c,
        "dtype": str(s.dtype),
        "n_unique": int(s.nunique(dropna=True)),
        "n_missing": int(s.isna().sum()),
        "pct_missing": float(s.isna().mean()*100.0),
    })

import pandas as pd
meta_df = pd.DataFrame(meta).sort_values("pct_missing", ascending=False)


REPORTS_DIR = Path("reports")
REPORTS_DIR.mkdir(parents=True, exist_ok=True)
meta_df.to_csv(REPORTS_DIR / "02_meta_missing_before.csv", index=False)

meta_df.head(12)


,column,dtype,n_unique,n_missing,pct_missing
48,COMMONAREA_AVG,float64,1108,8712,70.331799
62,COMMONAREA_MODE,float64,1093,8712,70.331799
76,COMMONAREA_MEDI,float64,1107,8712,70.331799
84,NONLIVINGAPARTMENTS_MEDI,float64,67,8648,69.815129
70,NONLIVINGAPARTMENTS_MODE,float64,48,8648,69.815129
56,NONLIVINGAPARTMENTS_AVG,float64,109,8648,69.815129
86,FONDKAPREMONT_MODE,object,4,8536,68.910955
54,LIVINGAPARTMENTS_AVG,float64,621,8531,68.870590
82,LIVINGAPARTMENTS_MEDI,float64,503,8531,68.870590
68,LIVINGAPARTMENTS_MODE,float64,409,8531,68.870590



## Cleaning rules (base)
- **Sentinel `DAYS_EMPLOYED == 365243` → NaN** + flag `DAYS_EMPLOYED_ANOM` (1/0).  
- **`DAYS_*`** bernilai negatif (menghitung mundur) → buat fitur turunan: `AGE_YEARS = -DAYS_BIRTH/365.25`, `EMP_YEARS = -DAYS_EMPLOYED/365.25`.  
- **`CODE_GENDER == 'XNA'` → NaN** (kategori tidak diketahui).  
- **Rasio kemampuan bayar**: `R_CREDIT_INCOME = AMT_CREDIT / AMT_INCOME_TOTAL`, `R_ANNUITY_INCOME = AMT_ANNUITY / AMT_INCOME_TOTAL` (dengan penangkal pembagi nol).  
- **Tipe hemat memori** untuk TARGET (`int8`) dan flag-flag (opsional).


In [5]:

import numpy as np
import pandas as pd

def preprocess_base(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    if "TARGET" in df.columns:
        df["TARGET"] = df["TARGET"].astype("int8")
    if "SK_ID_CURR" in df.columns:
        df["SK_ID_CURR"] = df["SK_ID_CURR"].astype("int64")

    if "DAYS_EMPLOYED" in df.columns:
        anom_mask = df["DAYS_EMPLOYED"] == 365243
        if "DAYS_EMPLOYED_ANOM" not in df.columns:
            df["DAYS_EMPLOYED_ANOM"] = (anom_mask).astype("int8")
        df.loc[anom_mask, "DAYS_EMPLOYED"] = np.nan

    if "DAYS_BIRTH" in df.columns:
        df["AGE_YEARS"] = (-df["DAYS_BIRTH"] / 365.25).astype("float32")
    if "DAYS_EMPLOYED" in df.columns:
        df["EMP_YEARS"] = (-df["DAYS_EMPLOYED"] / 365.25).astype("float32")

    if "CODE_GENDER" in df.columns:
        df["CODE_GENDER"] = df["CODE_GENDER"].replace({"XNA": np.nan})

    def safe_div(a, b):
        return a / np.where(b==0, np.nan, b)

    if {"AMT_CREDIT","AMT_INCOME_TOTAL"}.issubset(df.columns):
        df["R_CREDIT_INCOME"] = safe_div(df["AMT_CREDIT"], df["AMT_INCOME_TOTAL"]).astype("float32")
    if {"AMT_ANNUITY","AMT_INCOME_TOTAL"}.issubset(df.columns):
        df["R_ANNUITY_INCOME"] = safe_div(df["AMT_ANNUITY"], df["AMT_INCOME_TOTAL"]).astype("float32")

        if c.startswith("FLAG_") or c.endswith("_FLAG"):
            try:
                df[c] = df[c].astype("float").astype("Int8")
            except Exception:
                pass

    return df


In [6]:

df_clean = preprocess_base(df)

check_cols = [c for c in ["DAYS_EMPLOYED","DAYS_EMPLOYED_ANOM","AGE_YEARS","EMP_YEARS",
                          "R_CREDIT_INCOME","R_ANNUITY_INCOME","CODE_GENDER"] if c in df_clean.columns]
df_clean[check_cols].head(8)


,DAYS_EMPLOYED,DAYS_EMPLOYED_ANOM,AGE_YEARS,EMP_YEARS,R_CREDIT_INCOME,R_ANNUITY_INCOME,CODE_GENDER
0,-688.0,0,32.205338,1.883641,3.769231,0.110646,M
1,NaN,1,63.515400,NaN,2.666667,0.133333,F
2,-3432.0,0,52.470909,9.396304,2.666667,0.133333,F
3,-1154.0,0,28.517454,3.159480,2.500000,0.115850,F
4,-3256.0,0,33.196442,8.914442,6.349206,0.182429,F
5,-5896.0,0,36.427105,16.142368,2.259980,0.095740,F
6,-653.0,0,39.044491,1.787817,2.000000,0.100000,F
7,NaN,1,62.239563,NaN,3.127700,0.133950,F


In [7]:
from pathlib import Path
import pandas as pd

# daftar kolom yang kita pengen bandingin
cols_interest = [
    "DAYS_EMPLOYED",
    "DAYS_EMPLOYED_ANOM",
    "AGE_YEARS",
    "EMP_YEARS",
    "R_CREDIT_INCOME",
    "R_ANNUITY_INCOME",
    "CODE_GENDER",
]

# kolom mana yang ada di df (before) dan df_clean (after)
before_cols = [c for c in cols_interest if c in df.columns]
after_cols  = [c for c in cols_interest if c in df_clean.columns]

# hitung missing rate per kolom (before vs after)
before_tbl = (
    df[before_cols]
    .isna()
    .mean()
    .to_frame("pct_missing_before")
)

after_tbl = (
    df_clean[after_cols]
    .isna()
    .mean()
    .to_frame("pct_missing_after")
)

# gabung keduanya by index (nama kolom)
compare = before_tbl.join(after_tbl, how="outer")

# urutkan biar enak dibaca
compare = compare.sort_values(
    by=["pct_missing_after", "pct_missing_before"],
    ascending=False
)

# simpan artefak
Path("reports").mkdir(parents=True, exist_ok=True)
compare.to_csv(Path("reports") / "02_missing_compare_subset.csv")

compare.head(12)


,pct_missing_before,pct_missing_after
DAYS_EMPLOYED,0.0,0.186163
EMP_YEARS,NaN,0.186163
R_ANNUITY_INCOME,NaN,0.000081
CODE_GENDER,0.0,0.000000
AGE_YEARS,NaN,0.000000
DAYS_EMPLOYED_ANOM,NaN,0.000000
R_CREDIT_INCOME,NaN,0.000000


In [8]:
import sys; print(sys.executable)  # harus menunjuk ke ...\.venv\Scripts\python.exe


c:\Users\felix\hci_final_project\.venv\Scripts\python.exe


In [9]:
INTERIM_DIR = DATA_DIR / "interim"
INTERIM_DIR.mkdir(parents=True, exist_ok=True)

use_path = SAMPLE_PATH if SAMPLE_PATH.exists() else RAW_PATH
out_path = INTERIM_DIR / ("application_train_sample_clean.parquet" if use_path==SAMPLE_PATH else "application_train_clean.parquet")
df_clean.to_parquet(out_path, index=False)
out_path, df_clean.shape


(WindowsPath('C:/Users/felix/hci_final_project/data/interim/application_train_sample_clean.parquet'),
 (12387, 127))


### Next (di notebook 03 — Modeling)
- Gunakan `ColumnTransformer` + `Pipeline` untuk **imputasi median** (numeric) & **OneHotEncoder** (kategori) langsung di *fit*.  
- Evaluasi dengan **StratifiedKFold** dan metrik **ROC-AUC, PR-AUC, KS**.  
- Simpan **probabilitas** untuk simulasi **approve vs bad-rate**.

> Keuntungan: cleaning tahap ini **tidak** melakukan imputasi global → menghindari *data leakage*. Semua imputasi/encoding final dilakukan **per fold** saat training.
